In [1]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = "add your serpapi key here"

In [3]:
import warnings
warnings.filterwarnings('ignore')

### LLM

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)


"Maharaja's Palace: Fine Indian Cuisine" 


In [5]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Saffron Palace"'

### Prompt Templates 

In [7]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


"La Bella Cucina" (The Beautiful Kitchen)


### Chains 

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n1. "El Mirador" (The Overlook)\n2. "La Cantina de Oro" (The Golden Cantina)\n3. "El Sabor Mexicano" (The Mexican Flavor)\n4. "Casa Azul" (Blue House)\n5. "Rancho del Sol" (Ranch of the Sun)\n6. "Hacienda del Mar" (Estate of the Sea)\n7. "La Frontera" (The Border)\n8. "El Dorado Cocina" (The Golden Kitchen)\n9. "Mi Tierra" (My Land)\n10. "La Fiesta Mexicana" (The Mexican Party)'

In [11]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"El Jardín de Sabores" (The Garden of Flavors)'

In [12]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

### Simple Sequential Chain 

In [13]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Spicy Chicken Tikka Masala
2. Vegetable Biryani
3. Lamb Vindaloo
4. Tandoori Shrimp Skewers
5. Chana Masala (Chickpea Curry)
6. Aloo Gobi (Potato and Cauliflower Curry)
7. Paneer Tikka (Grilled Indian Cheese)
8. Samosas (Crispy Pastry Stuffed with Spiced Potatoes and Peas)
9. Naan Bread with Garlic and Chili Flakes
10. Mango Lassi (Refreshing Yogurt Drink)
11. Chicken Korma (Creamy Coconut and Cashew Curry)
12. Palak Paneer (Spinach and Indian Cheese Curry)
13. Butter Chicken (Tender Chicken in a Creamy Tomato Sauce)
14. Masala Dosa (Thin Crepe filled with Spiced Potatoes and Onions)
15. Gulab Jamun (Sweet Fried Dumplings in Syrup)


### Sequential Chain 

In [14]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [15]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [16]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [17]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Kingdom" ',
 'menu_items': '\n\n1. Spicy Chicken Tikka Masala\n2. Fiery Lamb Vindaloo\n3. Spicy Szechuan Beef\n4. Hot and Sour Soup\n5. Prawn Curry with Chili and Garlic\n6. Spicy Tandoori Shrimp\n7. Red Curry Chicken\n8. Spicy Hunan Tofu\n9. Thai Basil Fried Rice with Spicy Chicken\n10. Spicy Chicken Korma\n11. Mapo Tofu with Spicy Ground Pork\n12. Spicy Sichuan Noodles\n13. Spicy Garlic Naan Bread\n14. Korean Spicy BBQ Pork Ribs\n15. Cajun Spiced Prawns with Mango Salsa.'}

### Agents 

In [18]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool 

In [23]:
#!pip install serpapi

In [36]:
#!pip install google-search-results
#!pip install --upgrade google-search-results

In [46]:
#import serpapi
#print(dir(serpapi))

In [45]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# Load only the 'llm-math' tool for now, as there seems to be an issue with 'serpapi'
tools = load_tools(["llm-math"], llm=llm)

# Initialize the agent with the loaded tools and the OpenAI LLM
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Run a test query with the agent
response = agent.run("What was the GDP of the US in 2022 plus 5?")
print(response)



> Entering new AgentExecutor chain...
 I need to add 5 to the GDP of the US in 2022
Action: Calculator
Action Input: GDP of the US in 2022, 5
Observation: Answer: 10110
Thought: I now know the final answer
Final Answer: 10110

> Finished chain.
10110


#### Wikipedia and llm-math tool 

In [47]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman and investor known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. Other involvements include ownership of X Corp., the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of The Boring Company, xAI, Neuralink, and OpenAI. He is one of the wealthiest individuals in the world; as of August 2024 Forbes estimates his net worth to be US$247 billion.
Musk was born in Pretoria to Maye (née Haldeman), a model, and Errol Musk, a businessman and engineer. Musk briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later he matriculated at Queen's Uni

'Elon Musk was born in 1971 and will be 52 years old in 2023.'

### Memory 

In [48]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [49]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [50]:
chain.memory

In [51]:
type(chain.memory)

NoneType

### ConversationBufferMemory 

In [52]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [53]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [54]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


### ConversationChain 

In [55]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [56]:
convo.run("Who won the first cricket world cup?")

" The first cricket World Cup was held in 1975, and it was won by the West Indies team. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London. The West Indies team won by 17 runs, with Clive Lloyd being named as the Man of the Match. The World Cup was organized by the International Cricket Council (ICC) and featured eight teams, including Australia, England, India, Pakistan, New Zealand, Sri Lanka, and East Africa. "

In [57]:
convo.run("How much is 5+5?")

'  The answer to 5+5 is 10. This is a basic arithmetic equation that involves adding two numbers together. In this case, the numbers are both 5 and when added together, they equal 10. This is a fundamental concept in mathematics and is often taught in early education.'

In [58]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning West Indies team in the first cricket World Cup was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler, and he captained the West Indies team from 1974 to 1985. Lloyd was known for his aggressive and powerful batting style and his strong leadership skills. He went on to lead the West Indies team to win two more cricket World Cups, in 1979 and 1983. After his retirement, he became a coach and administrator for cricket teams around the world.'

In [59]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket World Cup was held in 1975, and it was won by the West Indies team. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London. The West Indies team won by 17 runs, with Clive Lloyd being named as the Man of the Match. The World Cup was organized by the International Cricket Council (ICC) and featured eight teams, including Australia, England, India, Pakistan, New Zealand, Sri Lanka, and East Africa. 
Human: How much is 5+5?
AI:   The answer to 5+5 is 10. This is a basic arithmetic equation that involves adding two numbers together. In this case, the numbers are both 5 and when added together, they equal 10. This is a fundamental concept in mathematics and is often taught in early education.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning West Indies team in the first cricket World Cup was Clive Lloyd. He was a left-handed batsman and a medium-pace 

### ConversationBufferWindowMemory 

In [60]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies team in 1975. It was held in England and the West Indies defeated Australia in the final match. The West Indies team was led by Clive Lloyd and they won by 17 runs. The man of the match was Viv Richards, who scored 138 runs in the final.'

In [61]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [62]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to accurately answer that question. Can you provide me with more information?'